In [6]:

from models import *
from config import *

from datasets import *

import math

dataset = ToyData("train")

idx = [4, 7, 9, 11]
ims = torch.cat([dataset[i]["image"].unsqueeze(0) for i in idx])

print(ims.shape)

percept = PSGNet(config.imsize,3)


outputs = percept(ims)

def normalize_outputs(outputs):
    recons = outputs["recons"]
    B = recons[0].shape[0]
    W = int(math.sqrt(recons[0].shape[1]))

    clusters  = outputs["clusters"]
    features  = outputs["features"]
    centroids = outputs["centroids"]
    moments   = outputs["moments"]

    for item in clusters:print(item[0].shape, item[1].shape)

    level_reconstructions = [item.reshape([B,W,W,3]) for item in recons]

    return {"recons":level_reconstructions, 
    "clusters": clusters, 
    "features": features, 
    "centroids": centroids, 
    "moments": moments,
    "batch":outputs["batch"]}

outputs = normalize_outputs(outputs)
recons = outputs["recons"]

for i,item in enumerate(recons):print("level:{}".format(i),item.shape)



class AbstractNet(nn.Module):
    def __init__(self,config):
        super().__init__()
        
        self.num_heads =8
        self.feature_decoder = nn.Transformer(nhead=16, num_encoder_layers=12,d_model = config.global_feature_dim,batch_first = True)
        self.spatial_decoder = nn.Transformer(nhead=16, num_encoder_layers=12,d_model = config.global_feature_dim,batch_first = True)
        self.source_heads = nn.Parameter(torch.randn([self.num_heads,config.global_feature_dim]))

        self.coordinate_decoder = nn.Linear(config.global_feature_dim, 2)
    
    def forward(self, feature, spatial):
        B, M, C = feature.shape
        N = self.num_heads
        # [Feature Propagation]
        component_features = feature
        component_spaitals = spatial

        # [Decode Proposals]
        global_feature = torch.randn()
        source_heads = self.souce_heads
        feature_proposals = self.feature_decoder(source_heads,global_feature)
        spaital_proposals = self.spatial_decoder(source_heads,global_feature)

        # [Component Matching]
        # component_features : [B,M,C]
        # feature_proposals  : [B,N,C]

        match = torch.softmax(torch.einsum("bnc,bmc -> bnm",component_features, proposal_features)/math.sqrt(C), dim = -1)
        existence = torch.max(match, dim = 1).values  # [B, N, 1]

        # [Construct Representation]
        output_graph = 0

        return output_graph

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/melkor/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


torch.Size([4, 128, 128, 3])
torch.Size([65536]) torch.Size([65536])
torch.Size([2198]) torch.Size([2198])
level:0 torch.Size([4, 128, 128, 3])
level:1 torch.Size([4, 128, 128, 3])


In [7]:
print("clusters")
clusters = outputs["clusters"]
for item in clusters:
    nodes = item[0]
    batch = item[1]
    print(nodes.shape, batch.shape)


print("features")
features = outputs["features"]
for item in features:
    print(item.shape)


print("centroids")
features = outputs["centroids"]
for item in features:
    print(item.shape)

print("moments")
features = outputs["moments"]
for item in features:
    print(item.shape)



clusters
torch.Size([65536]) torch.Size([65536])
torch.Size([2198]) torch.Size([2198])
features
torch.Size([2198, 64])
torch.Size([1021, 64])
centroids
torch.Size([2198, 2])
torch.Size([1021, 2])
moments
torch.Size([2198, 2])
torch.Size([1021, 2])


In [16]:
def to_dense_features(outputs):
    level_batch = outputs["batch"]
    features = outputs["features"]
    centroids = outputs["centroids"]
    moments = outputs["moments"]
    level_features = []
    for i in range(len(features)):
        sparse_feature = features[i]
        sparse_centroid = centroids[i]
        sparse_moment = moments[i]


        cast_batch = level_batch[i]

        feature,  batch = to_dense_batch(features[i],cast_batch)
        centroid, batch = to_dense_batch(centroids[i],cast_batch)
        moment,   batch = to_dense_batch(moments[i],cast_batch)


        level_features.append({"features":feature, "centroids":centroid, "moments":moment,"masks":batch.int()})
    return level_features



In [20]:
psg_features = to_dense_features(outputs)

base_graph = psg_features[-1]


In [87]:
import torch
import torch.nn as nn

width = 7
dim = 64

num_heads = width
feature_dim = dim
        
relation_extractor = None
propoerty_extractor = None

proposal_decoder = nn.Transformer(nhead=16, num_encoder_layers=12,d_model = feature_dim,batch_first = True, dim_feedforward = 128)
spatial_decoder = nn.Linear(64,2)
feature_decoder = nn.Linear(64,64)

#nn.Transformer(nhead=16, num_encoder_layers=12,d_model = feature_dim,batch_first = True, dim_feedforward = 128)
source_heads = nn.Parameter(torch.randn([num_heads,feature_dim]))

# [Test the input feature nodes]

input_graph = base_graph

# [Feature Propagation]
features = input_graph["features"]
spatials = input_graph["centroids"]
masks = input_graph["masks"]
C = 1
print(masks.shape)

# [Decode Proposals]
B = 4
source_key = source_heads.unsqueeze(0).repeat(B,1,1)
target_key = torch.randn([4, 25, 64])
proposals = proposal_decoder(source_key, target_key)
feature_proposals = feature_decoder(proposals)
spatial_proposals = spatial_decoder(proposals)

# [Component Matching]
 # component_features : [B,M,C]
component_features = torch.cat([features, spatials], -1)

# feature_proposals  : [B,N,C]
proposal_features = torch.cat([feature_proposals,spatial_proposals], -1)

print(component_features.shape, proposal_features.shape)

match = torch.softmax(torch.einsum("bnc,bmc -> bnm",component_features, proposal_features)/math.sqrt(C), dim = -1)
match = match * (masks.unsqueeze(-1))


existence = torch.max(match, dim = 1).values  # [B, N, 1]
print(match.shape)
# [Construct Representation]
output_graph = {"features":0, "masks":existence, "edge":match}

print(match.max())
print(match.min())
print(torch.sum(match, dim = 1))

torch.Size([4, 260])
torch.Size([4, 260, 66]) torch.Size([4, 25, 66])
torch.Size([4, 260, 25])
tensor(0.3086, grad_fn=<MaxBackward1>)
tensor(0., grad_fn=<MinBackward1>)
tensor([[11.1965,  4.4657, 12.1617,  2.1431,  7.2188,  8.1599,  3.0704,  1.7406,
          2.2825,  1.8195,  3.8319,  7.6983,  1.8318,  6.9286, 17.4582, 24.1367,
          5.1656,  4.3672,  5.2796,  1.5961, 10.1041,  2.1134,  6.0262,  7.4301,
         32.7737],
        [ 1.9365, 20.3519,  9.6738,  1.1250,  4.8740,  7.4356,  8.5526,  3.7956,
          3.9474,  4.1809,  7.5315,  2.1582,  3.9368,  6.7084, 12.8178,  3.0392,
         11.4981,  4.0942,  1.5633,  4.9882,  3.0433,  1.1033,  2.9720,  3.1339,
         49.5383],
        [16.4136,  3.0711,  3.6567,  2.3932,  5.6319,  3.0945, 10.4118,  3.9161,
          3.0266,  4.4475,  1.5872,  5.2647,  8.0425,  5.9415,  2.7761,  5.0957,
          9.7911, 14.1159,  1.4987, 30.9360,  3.3030, 21.5039, 18.4708,  4.5584,
          6.0515],
        [ 3.8527,  1.9679,  8.8792,  7.6273, 

In [88]:
a = [4.1488,  5.6714, 11.6582,  8.1207,  3.8911, 10.2980, 10.6350,  4.7643,
          5.7072,  6.0902,  7.7078, 12.6339,  3.3813, 12.5737, 24.0381, 16.0200,
         23.8276,  5.9760, 11.7249, 20.5651,  7.2184,  5.2609,  4.2785, 28.6767,
          5.1321]

cnt = 0
for i in a:cnt += i
print(cnt)

259.9999


In [89]:
import torch
import torch.nn.functional as F
import numpy as np
import torch.nn as nn
import os
import time
from torch_sparse import SparseTensor
import matplotlib.pyplot as plt


class GraphPropagation(nn.Module):
    def __init__(self,
                 num_iters=15,
                 excite=True,
                 inhibit=True,
                 project=False,
                 adj_thresh=0.5):
        super().__init__()
        self.num_iters = num_iters
        self.excite = excite
        self.inhibit = inhibit
        self.project = project
        self.adj_thresh = adj_thresh

        self.adj_e = None
        self.adj_i = None
        self.norm_factor_e = None
        self.norm_factor_i = None
        self.activation_converge = False

    def preprocess_inputs(self, h0, adj, activated):
        B, N, K = h0.shape

        # activated: binary tensor indicating nodes to start propagation from
        if activated is None: # randomly initialized at a point
            rand_idx = torch.randint(0, N, [B, ]).to(h0.device)
            activated = F.one_hot(rand_idx, num_classes=N).unsqueeze(-1).float()  # [BT, N, 1]
        else:
            activated = activated.reshape(B, N, 1)

        # create excitatory and inhibitory affinities, followed by thresholding
        if not isinstance(adj, SparseTensor): # dense tensor
            adj = adj.reshape(B, N, N)
            adj_e, adj_i = adj, 1.0 - adj
            if self.adj_thresh is not None:
                adj_e = self._threshold(adj_e, self.adj_thresh)
                adj_i = self._threshold(adj_i, self.adj_thresh)
            sample_mask = None
        else: # sparse tensor
            adj_e = adj
            adj_i = adj.copy().set_value_(1.0 - adj.storage.value())

            sample_mask = adj_e.copy()
            sample_mask = sample_mask.set_value_(torch.ones_like(sample_mask.storage.value()))

            if self.adj_thresh is not None:
                adj_e = self._threshold_sparse_tensor(adj_e, self.adj_thresh)
                adj_i = self._threshold_sparse_tensor(adj_i, self.adj_thresh)

        return h0, adj_e, adj_i, activated, sample_mask

    def forward(self, h0, adj, activated=None):
        """
        Function: Graph propagation to create the plateau map representation
        Input:
        - h0: initial hidden states (or equivalently, plateau map representation) with shape [B, N Q]
        - adj: affinity matrix with shape
        - activated: The graph propagation will start from the activated nodes
                   It can be None or binary tensor of shape [B, N].
                   If None, one node will be randomly selected as being activated
        Return:
        - plateau_map_list: a list of plateau maps of len self.num_iters. Each plateau map has shape [B, N, Q]
        """

        h0, adj_e, adj_i, activated, sample_mask = self.preprocess_inputs(h0, adj, activated)

        h = h0.clone()
        plateau_map_list = []
        running_activated = activated
        self.activation_converge = False

        # start graph propagation
        for it in range(self.num_iters):
            h, activated, running_activated = \
                self.propagate(h, adj_e, adj_i, activated, running_activated, sample_mask, it)
            plateau_map_list.append(h.reshape(h0.shape))

        return plateau_map_list

    def propagate(self, h, adj_e, adj_i, activated, running_activated, sample_mask, iter):
        B, N, D = h.shape

        # Graph propagation starts at a subset of activated nodes
        # If self.activation_converge is False, i.e. not all nodes are activated, \
        #   we need to apply masking to the affinities and compute the normalization factor accordingly.
        # We do so until all the nodes are activated, i.e.  self.activation_converge == True

        if not self.activation_converge:
            if isinstance(adj_e, SparseTensor):
                # apply the activation mask on the affinity tensors
                adj_e = adj_e.mul(activated.flatten()[None])
                adj_i = adj_i.mul(activated.flatten()[None])
                sample_mask = sample_mask.mul(activated.flatten()[None])

            # compute the normalization factors
            if not isinstance(adj_e, SparseTensor):
                norm_factor_e = torch.sum(adj_e.abs() * activated, dim=-2, keepdim=True).clamp(min=1.0).detach()
                norm_factor_i = torch.sum(adj_i.abs() * activated, dim=-2, keepdim=True).clamp(min=1.0).detach()
            else:
                norm_factor_e = adj_e.sum(1).reshape(B, N, 1).clamp(min=1.0).detach()
                norm_factor_i = adj_i.sum(1).reshape(B, N, 1).clamp(min=1.0).detach()

            self.norm_factor_e = norm_factor_e # [B,1,N]
            self.norm_factor_i = norm_factor_i # [B,1,N]
            self.adj_e = adj_e
            self.adj_i = adj_i

            self.activation_converge = activated.sum() == (B * N)
        else: # no update is required if all the nodes are activated
            adj_e = self.adj_e
            adj_i = self.adj_i
            norm_factor_e = self.norm_factor_e
            norm_factor_i = self.norm_factor_i
            sample_mask = None

        # [Excitation]
        if self.excite:
            if not isinstance(adj_e, SparseTensor):
                e_effects = torch.matmul(h.permute(0, 2, 1), adj_e * activated) / norm_factor_e
                e_effects = e_effects.permute(0, 2, 1)
            else:
                e_effects = adj_e.matmul(h.reshape(B * N, D))
                e_effects = e_effects.reshape(B, N, D) / norm_factor_e
            h = h + e_effects

        # [Inhibition]
        if self.inhibit:
            if not isinstance(adj_e, SparseTensor):
                i_effects = torch.matmul(h.permute(0, 2, 1), adj_i * activated) / norm_factor_i
                i_effects = i_effects.permute(0, 2, 1)
            else:
                i_effects = adj_i.matmul(h.reshape(B * N, D))
                i_effects = i_effects.reshape(B, N, D) / norm_factor_i

            proj = self._projection(h, i_effects) if self.project else i_effects
            h = h - proj

        h = self._relu_norm(h)

        # [Update activated nodes]
        if activated.sum() < B * N:
            if not isinstance(adj_e, SparseTensor):
                receivers = torch.max(torch.where(adj_e > adj_i, adj_e, adj_i) * activated, dim=1, keepdim=False)[0] > 0.5 # [B,N]
            else:
                assert sample_mask is not None
                receivers = sample_mask.max(dim=1) > 0.5
                receivers = receivers.reshape(B, N)

            running_activated = running_activated + receivers.unsqueeze(-1).float()
            activated = running_activated.clamp(max=1.0).detach()

        return h, activated, running_activated

    @staticmethod
    def _threshold(x, thresh):
        return x * (x > thresh).float()

    @staticmethod
    def _threshold_sparse_tensor(x, thresh):
        row, col, value = x.coo()
        valid = value > thresh
        sparse_size = [x.size(0), x.size(1)]
        output = SparseTensor(row=row[valid],col=col[valid],value=value[valid],sparse_sizes=sparse_size)
        return output

    @staticmethod
    def _projection(v, u, eps=1e-12):
        u_norm = torch.sum(u * u, -1, keepdims=True)
        dot_prod = torch.sum(v * u, -1, keepdims=True)
        proj = (dot_prod / (u_norm + eps)) * u
        return proj

    @staticmethod
    def _relu_norm(x, relu=True, norm=True, eps=1e-16):
        x = F.relu(x) if relu else x
        x = F.normalize(x + eps, p=2.0, dim=-1, eps=max([eps, 1e-12])) if norm else x
        return x


In [ ]:
propagator = GraphPropagation(7)

